In [ ]:
from glob import glob
from ultralytics import YOLO
import os

In [2]:
model_for_sbd_mdt = YOLO('best_sbd_mdt.pt')

In [3]:
test_image_files = glob(os.path.join('testset1', 'images', '*.jpg'))

result_path = os.path.join('submission', 'result1.txt')

In [ ]:
def process_and_write_to_file(results):
    ticked_boxes = {
        'SBD1': '',
        'SBD2': '',
        'SBD3': '',
        'SBD4': '',
        'SBD5': '',
        'SBD6': '', 
        'MDT1': '',
        'MDT2': '',
        'MDT3': ''
    }

In [ ]:
for image_path in test_image_files:
    results = model_for_sbd_mdt (image_path)
    process_and_write_to_file(results)

    